In [4]:
with open('data.txt', 'r') as file:
    # Read the entire content of the file
    data = file.read()

# # Print the content
# print(data)


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer(data, padding=True, truncation=True, return_tensors="pt")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00


In [29]:
from datasets import Dataset

dataset = Dataset.from_dict({"text": [data]})


In [30]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1
})

In [31]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [32]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [33]:
# tokenized_dataset = tokenized_dataset.rename_column("labels", "label")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask"])

In [34]:
train_dataset = tokenized_dataset
eval_dataset = tokenized_dataset

In [35]:
training_args = TrainingArguments(
    output_dir='./results',  # Where to save the model
    evaluation_strategy="epoch",  # Evaluate every epoch
    per_device_train_batch_size=8,  # Training batch size
    per_device_eval_batch_size=8,  # Evaluation batch size
    num_train_epochs=2002,  # Number of training epochs
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [23]:
trainer.save_model('./results')  # Saves the model
tokenizer.save_pretrained('./results')  # Saves the tokenize

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json',
 './results/tokenizer.json')

In [19]:
!pip install huggingface_hub


In [21]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [24]:
from transformers import BertForSequenceClassification, BertTokenizer
model = BertForSequenceClassification.from_pretrained('./results')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model.push_to_hub("zulqarnain-kernel/Mokalma")
tokenizer.push_to_hub("zulqarnain-kernel/Mokalma")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zulqarnain-kernel/Mokalma/commit/eb411fa6db735c1b7ee6b2c82c4a17668f5d3f6c', commit_message='Upload tokenizer', commit_description='', oid='eb411fa6db735c1b7ee6b2c82c4a17668f5d3f6c', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "zulqarnain-kernel/Mokalma"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


input_text = "Let's call it 1981."

input_ids = tokenizer.encode(input_text, return_tensors="pt")


output = model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at zulqarnain-kernel/Mokalma and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


let's call it 1981.ratactuswon yong paudant cnorraoukhendranindenndenstarɴemiarnornoɴmerie decimalninenine dopinglang hari hari barre dopinggia harieme ﬁllusgree manisas dl cn dopingeral decimal downed nwa ponce nationale dopingresh decimal someonedonnhamgreeicogicgreegic handicaptouyedtmndenico fright frightystdersɴsparthorpethorpe dlbainndi decimal mn galacticburnbainwonsasthorpenalrnoyedigen barretouwonɴ


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
import torch

def load_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text_data = file.read()
    return text_data

text_data = load_text_from_file('data.txt')

def split_text(text, chunk_size=1024):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

chunks = split_text(text_data)

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokens = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=1024)
    tokens["labels"] = tokens["input_ids"].copy()  # Labels should be the same as input_ids
    return tokens


data = {"text": chunks}
dataset = Dataset.from_dict(data)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,  # Adjust according to GPU memory
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,  # Only compute the loss
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)


trainer.train()

model.save_pretrained('./fine-tuned-gpt2')
tokenizer.save_pretrained('./fine-tuned-gpt2')


Map:   0%|          | 0/222 [00:00<?, ? examples/s]

In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel


model = GPT2LMHeadModel.from_pretrained("zulqarnain-kernel/mokalma-gpt2")  # Load from Hugging Face Hub
tokenizer = GPT2Tokenizer.from_pretrained("zulqarnain-kernel/mokalma-gpt2")  # Load from Hugging Face Hub


def generate_text(prompt, max_length=1028, num_return_sequences=1):

    input_ids = tokenizer.encode(prompt, return_tensors='pt')


    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=1.0
    )

    # Decode and print the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Example usage
prompt = "I don't know anything about it."
generated_text = generate_text(prompt, max_length=1028)
print("Generated Text:\n", generated_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text:
 I don't know anything about it.

   We stay in a taxi for a while, but after about 30 mins or so we realize it's too dangerous to get out. So we run down
  into the street, see nobody but the guy we started the taxi with,
two bouncers and two kids playing on the subway car's front
door. And then--


47  EXT. CITY HOSPITAL - MORNING  (TECHNICAL PLATFORM)  47  INT. SUBWAY HALL - LATER 47

 Joker steps out of his apartment and walks across the crowded
tiled floor, toward the small TV set where he lives. The sound of a
reminder machine buzzes through the narrow hallway-- 

48  BLAM! Out of nowhere, a UPS driver falls off a balcony-- Dead
32  in all! Joker grabs his bag and goes toward--
49  FLASHBACK! Three cops on horseback running down the alley behind
50  BOBBY SHORT (SAVING) TRAFFIC LAND, chasing after him-- CLOSE ON JOKER, blood
paint smearing his face. 




In [2]:
prompt = "Joker asked"
generated_text = generate_text(prompt, max_length=1028)
print("Generated Text:\n", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 Joker asked if I was interested in writing.
   It was a personal matter and he said, "I don't really have
a story yet."

  His mom looks at him, and as he starts to fill in the blank,
laughter subsides.


95  INT. MOM'S APARTMENT, LIVING ROOM - DAY 95  95

 Joker is alone in her apartment. She's dressed in a
silky white, faded, worn-out red bathrobe and
clothes line her face. Her hair's slicked back. This isn't a big deal. It's
the apartment she used to live in when she was young. A cramped
room, filthy floor, a TV in one corner and two small
lights that can't be turned on. The only sound is the harsh rustling of
her hair, the rustle of her shoes. 

 95 of 96 
